# Circuit Decomposition 
Edge and gate decomposition with zero classical communication.

Prerequisites: 

1. Create a new Conda environment
```shell
conda env create -n azure-knitting
```

1. Install everything needed to run the `modified-circuit-knitting-toolbox` locally: 
```shell
cd modified-circuit-knitting-toolbox
pip install -e .
```

1. Install everything needed to run on Azure:
```shell
python -m pip install --upgrade 'azure-quantum[qiskit]' qsharp ipykernel
pip install qiskit-aer==0.12.1
```

### Documentation
Running on Azure: [instructions](https://learn.microsoft.com/en-us/azure/quantum/quickstart-microsoft-qiskit?tabs=tabid-quantinuum&pivots=platform-local#prerequisites)


First, we enable communication with our Azure instance

In [1]:
# Prevent from overriding already run experiments
# exit()

In [2]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider


workspace = Workspace(
            resource_id = "/subscriptions/4ed1f7fd-7d9e-4c61-9fac-521649937e65/resourceGroups/Cutting/providers/Microsoft.Quantum/Workspaces/Cutting",
            location = "eastus")


provider = AzureQuantumProvider(workspace)

Check what Azure targets are available. Note that Quantinuum may need to be enabled in your workspace.

In [3]:
print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

This workspace's targets:
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-2e
- quantinuum.sim.h1-2e
- quantinuum.qpu.h1-1
- quantinuum.qpu.h1-1
- quantinuum.qpu.h1-2
- quantinuum.qpu.h1-2


In [4]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import EfficientSU2

# Create a quantum circuit to cut. We create a simple ansatz
mapper = JordanWignerMapper()
ansatz = EfficientSU2(3, reps=1)
# Decompose to the actual individual gates
circuit = ansatz.decompose(reps=3)
# Set some arbitrary parameters
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)

print(circuit)

global phase: 3.8832
     ┌─────────────┐┌─────────────┐                    ┌─────────────┐»
q_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├────────────■───────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤          ┌─┴─┐     ├─────────────┤»
q_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──■───────┤ X ├─────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤┌─┴─┐┌────┴───┴────┐├─────────────┤»
q_2: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ X ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├»
     └─────────────┘└─────────────┘└───┘└─────────────┘└─────────────┘»
«     ┌─────────────┐
«q_0: ┤ U3(0,0,0.8) ├
«     ├─────────────┤
«q_1: ┤ U3(0,0,0.8) ├
«     └─────────────┘
«q_2: ───────────────
«                    


Create a list of observables to use, and automatically find cut locations.

In [5]:
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import cut_wires_and_gates_to_subcircuits

observables = ["ZZI", "IZZ", "IIZ", "XIX", "ZIZ", "IXI"]

subcircuits, subobservables = cut_wires_and_gates_to_subcircuits(
    circuit=circuit,
    observables=observables,
    method='automatic',
    max_subcircuit_width=2,
    max_cuts=9,
    num_subcircuits=[2]
)

ModuleNotFoundError: No module named 'circuit_knitting'

In [ ]:
# Visualize the subcircuits. Note the decomposed 2-qubit gates marked 'cut_cx_0'
for key in subcircuits.keys():
    print(subcircuits[key])

       ┌─────────────┐┌─────────────┐                 ┌─────────────┐»
q11_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──────────────■──┤ U3(0.8,0,0) ├»
       ├─────────────┤├─────────────┤┌──────────┐┌─┴─┐├─────────────┤»
q11_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ X ├┤ U3(0.8,0,0) ├»
       └─────────────┘└─────────────┘└──────────┘└───┘└─────────────┘»
«       ┌─────────────┐
«q11_0: ┤ U3(0,0,0.8) ├
«       ├─────────────┤
«q11_1: ┤ U3(0,0,0.8) ├
«       └─────────────┘
     ┌─────────────┐┌─────────────┐┌──────────┐┌─────────────┐┌─────────────┐
q12: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├
     └─────────────┘└─────────────┘└──────────┘└─────────────┘└─────────────┘


In [ ]:
from circuit_knitting.cutting.gate_and_wire_cutting.evaluation import azure_queue_experiments

# Use the actual QPU backend
backend = 'quantinuum.sim.h1-1e'

# Submit the subcircuits to Azure Quantum
job_list, qpd_list, coefficients, subexperiments = azure_queue_experiments(
    circuits=subcircuits,
    subobservables=subobservables,
    num_samples=8,  # 8 unique samples to get some statistics
    backend = backend,
    provider = provider,
    shots=128  # Balance of cost and accuracy
)

Check on the actual results from run on Quantinuum.

In [ ]:
from circuit_knitting.cutting.gate_and_wire_cutting.evaluation import get_experiment_results_from_jobs

experiment_results = get_experiment_results_from_jobs(job_list, qpd_list, coefficients)
experiment_results

..............

KeyboardInterrupt: 

Now check the actual expectation values from the observables.

In [ ]:
from circuit_knitting.cutting.cutting_reconstruction import reconstruct_expectation_values

quantinuum_expvals = reconstruct_expectation_values(*experiment_results, subobservables)
quantinuum_expvals

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [ ]:
# Create ideal results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import exact_observables

ideal_expvals = exact_observables(circuit, observables)
ideal_expvals

array([0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229,
       0.23530298])

In [ ]:
# Compare the error between results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import compare_results

compare_results(quantinuum_expvals, ideal_expvals)

Simulated expectation values: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Exact expectation values: [0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229, 0.23530298]
Errors in estimation: [0.49609304, 0.42545843, 0.60415462, 0.90201184, 0.81518771, 0.76469702]
Relative errors in estimation: [0.98449331, 0.74051809, 1.52623889, 9.20531428, 4.41089546, 3.24984]


The below is a cost estimator for the circuits run - make sure to add up for all subcircuits run.

In [ ]:
cost_backend = provider.get_backend('quantinuum.qpu.h1-1')
total_cost = 0
for sample in subexperiments:
    for partition in sample:
        for subexperiment in partition:
            cost = cost_backend.estimate_cost(subexperiment, shots=128)
            total_cost += cost.estimated_total
print(f'Estimated cost: {total_cost} {cost.currency_code}')

Estimated cost: 133.8752 EHQC
